In [106]:
import numpy as np
import pandas as pd

X = pd.read_csv('data/data_synop_krakow.csv')
Y = pd.read_csv('data/data_pm10_krakow.csv')

In [107]:
X = X.sort_values(by=['date'], ascending=True)

for i in range(1, 8):
    X[f'minus_{i}_days_cloud'] = X['cloud'].shift(i)
    X[f'minus_{i}_days_wind'] = X['wind'].shift(i)
    X[f'minus_{i}_days_temp'] = X['temp'].shift(i)
    X[f'minus_{i}_days_humidity'] = X['humidity'].shift(i)
    X[f'minus_{i}_days_pressure'] = X['pressure'].shift(i)
    X[f'minus_{i}_days_precipitation_day'] = X['precipitation_day'].shift(i)
    X[f'minus_{i}_days_precipitation_night'] = X['precipitation_night'].shift(i)

X = X.drop(columns=['cloud', 'wind', 'temp', 'humidity', 'pressure', 'precipitation_day', 'precipitation_night'])
X = X.dropna()

X_1 = X.drop(columns=[column for column in list(X.columns) if any(str(i) in column for i in range(2, 8))])
XY_1 = X_1.merge(Y, on='date')
XY_1 = XY_1.drop(columns='date')
X_3 = X.drop(columns=[column for column in list(X.columns) if any(str(i) in column for i in range(4, 8))])
XY_3 = X_3.merge(Y, on='date')
XY_3 = XY_3.drop(columns='date')
X_7 = X.copy()
XY_7 = X_7.merge(Y, on='date')
XY_7 = XY_7.drop(columns='date')


In [108]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

X1 = XY_1.drop(['pm_10'], axis=1).values
Y1 = pd.cut(XY_1['pm_10'], bins=[-np.Inf, 50.0, 100.0, 150.0, np.Inf], labels=['clean', 'polluted', 'warning', 'alarm']).values
X3 = XY_3.drop(['pm_10'], axis=1).values
Y3 = pd.cut(XY_3['pm_10'], bins=[-np.Inf, 50.0, 100.0, 150.0, np.Inf], labels=['clean', 'polluted', 'warning', 'alarm']).values

X7 = XY_7.drop(['pm_10'], axis=1).values
Y7 = pd.cut(XY_7['pm_10'], bins=[-np.Inf, 50.0, 100.0, 150.0, np.Inf], labels=['clean', 'polluted', 'warning', 'alarm']).values

regr = RandomForestClassifier(n_jobs=8)
print(cross_validate(regr, X1, Y1, n_jobs=8)['test_score'])
print(cross_validate(regr, X3, Y3, n_jobs=8)['test_score'])
print(cross_validate(regr, X7, Y7, n_jobs=8)['test_score'])

[0.56751468 0.57860404 0.57142857 0.57637076 0.6194517 ]
[0.58643183 0.58969341 0.58773646 0.60052219 0.64295039]
[0.56686236 0.58121331 0.59817352 0.58420366 0.63577023]
